In [ ]:
import cv2
import pytesseract
import re



In [ ]:
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [ ]:


def detect_plate_number(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Better filtering
    filtered = cv2.bilateralFilter(gray, 11, 17, 17)
    edged = cv2.Canny(filtered, 30, 200)
    
    # Find contours
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
    
    plate_img = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 0.018 * cv2.arcLength(contour, True), True)
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = w / h
            if 2 < aspect_ratio < 6:  # Typical plate aspect ratio range
                plate_img = gray[y:y+h, x:x+w]
                cv2.drawContours(image, [approx], -1, (0, 255, 0), 3)
                break
    
    if plate_img is not None:
        # Adaptive threshold + OCR
        thresh = cv2.adaptiveThreshold(plate_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY, 11, 2)
        config = r'--oem 3 --psm 7'
        text = pytesseract.image_to_string(thresh, config=config)
        # Clean plate number with regex
        plate_number = re.findall(r'[A-Z0-9]{4,}', text.upper())
        return plate_number[0] if plate_number else "Not detected"
    else:
        return "No plate detected"

In [ ]:
image_path = "car.jpg" 

plate_number = detect_plate_number(image_path)
print("Detected Plate Number:", plate_number)